# Analytical Verification of Kalman filter and smoother
> 


We start with a simple verification for a model that only consists of $X_0, X_1, Y_0, Y_1$.

For this model the filtering and smoothing distributions is easily derived by hand and we can compare to the outputs of the `isssm.kalman.kalman`/`isssm.kalman.smoother` and `isssm.kalman.sqrt_filter`/`isssm.kalman.sqrt_smoother` methods.



In [ ]:
from isssm.glssm_models import lcm
from isssm.glssm import simulate_glssm
from isssm.kalman import kalman, smoother, sqrt_kalman, sqrt_smoother
import jax.numpy as jnp
import jax.random as jrn
import fastcore.test as fct
import jax.scipy.linalg as jsla

In [ ]:
x0, A, B, Sigma, Omega = lcm(1, 0., 1., 1., 1.)

_, (y,) = simulate_glssm(x0, A, B, Sigma, Omega, 1, jrn.PRNGKey(34234))


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
EX = jnp.array([x0, x0])
EY = jnp.array([x0, x0])

CovX = jnp.array([[1., 1.], [1., 2.]])
CovY = jnp.array([[2., 1.], [1., 3.]])

CovXY = jnp.array([[1., 1.], [1., 2.]])

EX_c_Y = EX + CovXY @ jnp.linalg.inv(CovY) @ (y - EY)
CovX_c_Y = CovX - CovXY @ jnp.linalg.inv(CovY) @ CovXY.T


In [ ]:
x_filt, Xi_filt, x_pred, Xi_pred = kalman(y, x0, Sigma, Omega, A, B)
x_smooth, Xi_smooth = smoother(x_filt, Xi_filt, x_pred, Xi_pred, A)

fct.test_close(x_smooth, EX_c_Y)
fct.test_close(Xi_smooth, jnp.diag(CovX_c_Y).reshape((2,1,1)))

fct.test_close(x_smooth[-1], x_filt[-1])
fct.test_close(Xi_smooth[-1], Xi_filt[-1])

In [ ]:
cuSigma = jsla.cholesky(Sigma)
cuOmega = jsla.cholesky(Omega)
cx_filt, cuXi_filt, cx_pred, cuXi_pred, G, cuH = sqrt_kalman(y, x0, cuSigma, cuOmega, A, B)

cx_smooth, cuXi_smooth = sqrt_smoother(cx_filt, cuXi_filt, cx_pred, cuXi_pred, G, cuH)

fct.test_close(Xi_smooth, cuXi_smooth**2)
fct.test_close(x_smooth, cx_smooth)

::: {.content-hidden}
Any GLSSM can also be written in matrix form (see [@Durbin2012Time] Section 4.13) and we will use this to analytically verify that filter, smoother and likelihood calculation work as expected.

Let $X = (X_0, \dots, X_n)$, $Y = (Y_0, \dots, Y_n)$, $\varepsilon = (\varepsilon_{1}, \dots, \varepsilon_{n})$ and $\eta = (\eta_0, \dots, \eta_n)$.
We can then express the relationships between $X,Y,\varepsilon,\eta$ as 

$$
\begin{align*}
    X &= A \left(\begin{pmatrix} X_0 \\ 
    \mathbf 0_{n \times 1} \end{pmatrix} + R \varepsilon\right)\\
    Y &= B X + \eta
\end{align*}
$$

with appropriate matrices $A, R, B$. The covariance matrices of $\varepsilon$ and $\eta$ are block-diagonal matrices containing $\Sigma_t$ and $\Omega_t$, $t = 0, \dots, n$.

:::

::: {.content-hidden}
Performing the Kalman smoother is then equivalent to calculating the conditional distribution of $X$ given $Y$, which is a normal distribution with mean
$$
\mathbf E \left(X | Y \right) = \E X + \cov (X, Y) \cov(Y)^{-1} (Y - \E Y)
$$
and covariance
$$
\cov (X | Y )= \cov (X) - \cov (X, Y) \cov(Y)^{-1} \cov(Y, X).
$$

In this setting we have 
$$
\begin{align*}
    A &= \begin{pmatrix} 
        I & 0 & 0 & 0 & \dots \\ 
        A_0 & I & 0 & 0 & \dots \\
        A_1 A_0 & A_1 & I & 0 & \dots \\
        \dots & \dots & \dots & \dots & \dots \\
     \end{pmatrix} \\
    R &=\begin{pmatrix} 
        0 & 0 & 0 & 0 & \dots \\ 
        I & 0 & 0 & 0 & \dots \\
        0 & I & 0 & 0 & \dots \\
        0 & 0 & I & 0 & \dots \\
        \dots & \dots & \dots & \dots & \dots \\
     \end{pmatrix} \\
     \cov {\varepsilon} &= \text{diag}(\Sigma_0, \dots, \Sigma_n) \\
    B &= \text{diag} (B_0, \dots, B_n) \\
    \cov{\eta} &= \text{diag} (\Omega_0, \dots, \Omega_n)
\end{align*} 
$$
:::